In [ ]:
import json
import csv
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt
import glob
import os
import datetime

In [ ]:
bids = pd.read_csv("../data/as_bids_v2.csv")
prices = pd.read_csv("../data/AS_price_vol.csv")
plan = pd.read_csv("../data/as_plan.csv")
energy = pd.read_csv("../data/energy_price.csv")
weather = pd.read_csv("../data/weather_forecast_ercot.csv")
gen = pd.read_csv("../data/generation.csv")
wind = pd.read_csv("../data/wind.csv")

In [ ]:
#bit more data cleaning
#generation
gen['hr_beg'] = gen['hr_beg'].apply(lambda x: x[0:2]) 
gen.drop(columns=['Imports','Other','ST'], inplace=True)

gen['year'] = pd.to_datetime(gen['date']).dt.year
gen = gen.loc[gen['year']>2011,:]
gen.reset_index(inplace=True, drop=True)
gen.drop(columns=['year'], inplace=True)

#energy
energy.drop_duplicates(subset=['date','hr_beg'], inplace=True)

In [ ]:
data_frames = [bids, prices, plan, energy, weather, gen, wind]

for i,df in enumerate(data_frames):
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['hr_beg'] = df['hr_beg'].astype(int)
    df.sort_values(by=['date','hr_beg'], inplace=True)
    df.drop_duplicates(inplace=True)
    #df.set_index(keys=['date','hr_beg'], inplace=True)
    

In [ ]:
#ugly merge, but loop fails me
union = prices.merge(bids, how='inner', on=['date','hr_beg'])
union = union.merge(plan, how='inner', on=['date','hr_beg'])
union = union.merge(weather, how='inner', on=['date','hr_beg'])
union = union.merge(gen, how='inner', on=['date','hr_beg'])
union = union.merge(energy, how='inner', on=['date','hr_beg'])
union = union.merge(wind, how='inner', on=['date','hr_beg'])

In [ ]:
#dealing with NAs
union.fillna(method='bfill',axis=0, inplace=True)

In [ ]:
union.sort_values(by=['date','hr_beg'], inplace=True)